## SOURCE TO BRONZE LAYER

### Process:

> The function fetches daily stock data using the Yahoo Finance API (`yfinance`).


In [1]:
# Import necessary libraries
import json
from datetime import timedelta

import pandas as pd
import requests
import yfinance as yf
from pyrate_limiter import Duration, Limiter, RequestRate
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket

from StockETL import DateTimeUtil, GlobalPath

In [2]:
# Importing Common Utility Function
%run ../COMMON/common_utility.ipynb

USERNAME = 'ptprashanttripathi'


In [3]:
# Define Constants file paths
stock_tickers_config_path = GlobalPath("CONFIG/CONSTANTS/stock_tickers.json")
holding_history_path = GlobalPath(f"DATA/SOURCE/Holding/{USERNAME}/Holding_data.csv")
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockdata_bronze_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/BRONZE/StockData.json"
)

In [4]:
# Setting cache location for yfinance
yf.set_tz_cache_location(".cache")


# Rate limiting setup
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass


history_rate = RequestRate(1, Duration.SECOND * 2)
limiter = Limiter(history_rate)
session = CachedLimiterSession(
    limiter=limiter,
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache(".cache/session", expire_after=timedelta(hours=1)),
)

In [5]:
def download_file_from_github(output_file):
    github_data_url = f"https://raw.githubusercontent.com/PtPrashantTripathi/PortfolioTracker/main/DATA/BRONZE/StockData/{output_file.name}"
    response = requests.get(github_data_url)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            file.write(response.content)
    else:
        raise Exception("Failed to download file from Github")
    print(f"Data processed and saved to => {output_file}")

In [6]:
def process_file(stock_ticker, date, output_file):
    """Fetch historical stock data and save it to a CSV file."""
    df = stock_ticker.history(
        start=date.start_date,
        end=min(date.end_date, DateTimeUtil.today()),
        interval="1d",
        actions=True,
        rounding=True,
    )
    if df.empty:
        raise Exception(f"No data fetched from {date.start_date} to {date.end_date}")
    df = df.reset_index()

    # Replace punctuation from column names for consistency
    df = replace_punctuation_from_columns(df)

    # Fix duplicate column names by appending numerical suffixes
    df = fix_duplicate_column_names(df)

    # Drop rows where all elements are NaN
    df = df.dropna(how="all")

    # Align Datafame with DataContract
    df = align_with_datacontract(df, stockdata_bronze_schema_file_path)

    df.to_csv(output_file, index=False)
    print(f"Data processed and saved to => {output_file}")

In [7]:
# Dictionary for stock ticker overrides
# Open and read the JSON file
OVERWRITE_TICKERS = {}
with open(stock_tickers_config_path, encoding="utf-8") as f:
    # Get the contract_fields from json data
    OVERWRITE_TICKERS = json.load(f)

In [8]:
# Load holding data from CSV
df_holding_history = pd.read_csv(holding_history_path)
df_holding_history["min_date"] = pd.to_datetime(df_holding_history["min_date"])
df_holding_history["max_date"] = pd.to_datetime(df_holding_history["max_date"])
print(f"Loaded data from => {holding_history_path}")

Loaded data from => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/SOURCE/Holding/ptprashanttripathi/Holding_data.csv


In [9]:
# Fetch stock data and process it.
for _, row in df_holding_history.iterrows():
    print(f"\nProcessing data for symbol {row['symbol']} =>")
    try:
        stock_ticker = yf.Ticker(
            OVERWRITE_TICKERS.get(row["symbol"], row["isin"]),
            session=session,
        )
        date_list = generate_date_list(
            row["min_date"].to_pydatetime(), row["max_date"].to_pydatetime()
        )
        for date in date_list:
            output_file = stockdata_bronze_layer_path.joinpath(
                f"{row['symbol']}_{date.year:04d}_{date.month:02d}.csv"
            )
            if (
                output_file.exists()
                and date.month_difference(DateTimeUtil.today()) >= 1
            ):
                continue
            else:
                try:
                    download_file_from_github(output_file)
                except:
                    process_file(stock_ticker, date, output_file)
    except Exception as e:
        print(f"Error processing {row['symbol']} =>\n{e}")


Processing data for symbol BAJAJHFL =>


Error processing BAJAJHFL =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol BHAGERIA =>

Processing data for symbol BPCL =>

Processing data for symbol GOLDBEES =>

Processing data for symbol HERANBA =>

Processing data for symbol IDEA =>

Processing data for symbol INFY =>

Processing data for symbol IRCTC =>


Error processing IRCTC =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol KPITTECH =>


Error processing KPITTECH =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol LICI =>

Processing data for symbol NIFTYBEES =>

Processing data for symbol PNB =>

Processing data for symbol SBIN =>

Processing data for symbol TATACHEM =>

Processing data for symbol TATAMOTORS =>


Error processing TATAMOTORS =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol TATAPOWER =>


Error processing TATAPOWER =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol UNITDSPR =>


Error processing UNITDSPR =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol VOLTAS =>

Processing data for symbol YESBANK =>

Processing data for symbol MIRAE_ASSET_ELSS_TAX_SAVER_FUND_DIRECT_PLAN_GROWTH =>


Error processing MIRAE_ASSET_ELSS_TAX_SAVER_FUND_DIRECT_PLAN_GROWTH =>
Too Many Requests. Rate limited. Try after a while.

Processing data for symbol SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH =>


Error processing SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH =>
Too Many Requests. Rate limited. Try after a while.
